In [1]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torch
from torchvision import transforms
import import_ipynb
from FaceRecognition.config import batch_size, rootDir, device, lr, epochs, base_output
from FaceRecognition.data import AFAD
from FaceRecognition.model import MTL
import os
from PIL import Image
import cv2
from pandas import DataFrame
import pandas as pd

importing Jupyter notebook from D:\ScORpioET\Face-recognition-and-detection\FaceRecognition\config.ipynb
importing Jupyter notebook from D:\ScORpioET\Face-recognition-and-detection\FaceRecognition\data.ipynb
importing Jupyter notebook from D:\ScORpioET\Face-recognition-and-detection\FaceRecognition\model.ipynb


In [2]:
img = cv2.imread('car.jpg')

In [3]:
Recognition_model = MTL().to(device=device)
Recognition_model.load_state_dict(torch.load('./FaceRecognition_model.pth'))
Recognition_model.eval()

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MTL(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Con

In [4]:
Detection_model = torch.hub.load('ultralytics/yolov5', 'custom', path='./FaceDetection_model.pt')
Bboxs = Detection_model(img)

Using cache found in C:\Users\x9172/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-3 Python-3.7.9 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [5]:
def drawBoundingBox(img, box):
    x1,y1,x2,y2 = (box['x1'], box['y1'], box['x2'], box['y2'])
    label = box['label']
    cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 6)
    fontFace = cv2.FONT_HERSHEY_COMPLEX
    fontScale = 0.5
    thickness = 1
    labelSize = cv2.getTextSize(label, fontFace, fontScale, thickness)
    _x1 = x1 # bottomleft x of text
    _y1 = y1 # bottomleft y of text
    _x2 = x1+labelSize[0][0] # topright x of text
    _y2 = y1-labelSize[0][1] # topright y of text
    cv2.rectangle(img, (_x1,_y1), (_x2,_y2), (0,255,0), cv2.FILLED) # text background
    cv2.putText(img, label, (x1,y1), fontFace, fontScale, (0,0,0), thickness)
    return img

In [6]:
if(len(Bboxs.pandas().xyxy[0]["name"].values)==0):
    print('no person')
else:
    for _, Bbox in Bboxs.pandas().xyxy[0].iterrows():
        bbox_xmin = int(Bbox["xmin"])
        bbox_xmax = int(Bbox["xmax"])
        bbox_ymin = int(Bbox["ymin"])
        bbox_ymax = int(Bbox["ymax"])

        capture = Image.fromarray(cv2.cvtColor(img[bbox_ymin:bbox_ymax,bbox_xmin:bbox_xmax], cv2.COLOR_BGR2RGB))

        trns = transforms.Compose([
                        transforms.Resize((32, 32)), 
                        transforms.ToTensor(), 
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        img_tensor = trns(capture).unsqueeze(0).to(device=device)

        age_output, gender_output = Recognition_model(img_tensor)

        age = round(age_output.item())
        gender_list = gender_output.tolist()[0]
        gender = 'male' if gender_list[0] >= gender_list[1] else 'female'

        box = {}
        box['label'] = f'Age:{age} Gender:{gender}'
        box['x1'] = bbox_xmin
        box['x2'] = bbox_xmax
        box['y1'] = bbox_ymin
        box['y2'] = bbox_ymax
        drawBoundingBox(img, box)

    cv2.imshow('Prediction', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

no person
